In [2]:
# -*- coding:utf-8 -*-
# Modified Author: Inyong Hwang (inyong1020@gmail.com)
# Date: 2019-08-06-Tue
# 파이토치 첫걸음 Chapter 4. 이미지 처리와 합성곱 신경망

import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

from torchvision.datasets import ImageFolder
from torchvision import transforms

# 4.5 DCGAN을 사용한 이미지 생성

# data: http://www.robots.ox.ac.uk/~vgg/data/flowers/102/

img_data = ImageFolder("./102flowers/",
                       transform=transforms.Compose([transforms.Resize(80),
                                                     transforms.CenterCrop(64),
                                                     transforms.ToTensor()]))

batch_size = 64
img_loader = DataLoader(img_data, batch_size=batch_size, shuffle=True)

In [30]:
nz = 100
ngf = 32

class GNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    
    def forward(self, x):
        out = self.main(x)
        return out

'''
out_size = (in_size - 1) * stride - 2 * padding + kernel_size + output_padding
in_size = 1
stride = 1
padding = 0
kernel_size = 4
output_padding = 0
'''

'\nout_size = (in_size - 1) * stride - 2 * padding + kernel_size + output_padding\nin_size = 1\nstride = 1\npadding = 0\nkernel_size = 4\noutput_padding = 0\n'

In [31]:
ndf = 32

class DNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False)
        )
    
    def forward(self, x):
        out = self.main(x)
        return out.squeeze()

In [32]:
d = DNet().to("cuda:0")
g = GNet().to("cuda:0")

opt_d = optim.Adam(d.parameters(), lr=0.0002, betas=(0.5, 0.999))
opt_g = optim.Adam(g.parameters(), lr=0.0002, betas=(0.5, 0.999))

ones = torch.ones(batch_size).to("cuda:0")
zeros = torch.zeros(batch_size).to("cuda:0")
loss_f = nn.BCEWithLogitsLoss()

fixed_z = torch.randn(batch_size, nz, 1, 1).to("cuda:0")

In [40]:
from statistics import mean

def train_dcgan(g, d, opt_g, opt_d, loader):
    log_loss_g = []
    log_loss_d = []
    for real_img, _ in tqdm.tqdm(loader):
        batch_len = len(real_img)
        
        real_img = real_img.to("cuda:0")
        
        z = torch.randn(batch_len, nz, 1, 1).to("cuda:0")
        fake_img = g(z)
        
        fake_img_tensor = fake_img.detach()
        
        out = d(fake_img)
        loss_g = loss_f(out, ones[: batch_len])
        log_loss_g.append(loss_g.item())
        
        d.zero_grad(), g.zero_grad()
        loss_g.backward()
        opt_g.step()
        
        real_out = d(real_img)
        loss_d_real = loss_f(real_out, ones[: batch_len])
        
        fake_img = fake_img_tensor
        
        fake_out = d(fake_img_tensor)
        loss_d_fake = loss_f(fake_out, zeros[: batch_len])
        
        loss_d = loss_d_real + loss_d_fake
        log_loss_d.append(loss_d.item())
        
        d.zero_grad(), g.zero_grad()
        loss_d.backward()
        opt_d.step()
        
    return mean(log_loss_g), mean(log_loss_d)

In [42]:
from torchvision.utils import save_image

for epoch in range(300):
    train_dcgan(g, d, opt_g, opt_d, img_loader)
    if epoch % 10 == 0:
        torch.save(
            g.state_dict(),
            "./Chapter_4.5_output/g_{:03d}.prm".format(epoch),
            pickle_protocol=4
        )
        torch.save(
            d.state_dict(),
            "./Chapter_4.5_output/d_{:03d}.prm".format(epoch),
            pickle_protocol=4
        )
        generated_img = g(fixed_z)
        save_image(
            generated_img,
            "./Chapter_4.5_output/{:03d}.jpg".format(epoch)
                  )

100%|████████████████████████████████████████████████████████████████████████████████| 128/128 [00:44<00:00,  2.91it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 128/128 [00:45<00:00,  2.82it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 128/128 [00:43<00:00,  2.94it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 128/128 [00:44<00:00,  2.89it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 128/128 [00:44<00:00,  2.89it/s]


In [37]:
# from IPython.display import Image, display_jpeg
# display_jpeg(Image('102flowers/000.jpg'))